# Transformaciones

## Leer múltiples archivos

In [92]:
# Importación de librerías
import boto3
import pandas as pd
from io import StringIO

In [93]:
# Después de importar boto3, se debe de indicar qué servicio o servicios se van a usar, en este caso s3.
s3 = boto3.resource('s3')

# Se accede al bucket llamado 'xetra-1234' disponible en s3.
bucket = s3.Bucket('xetra-1234')

# Se obtienen dos objetos dentro del bucket que cumplan con un criterio de fecha.
bucket_obj1 = bucket.objects.filter(Prefix='2022-01-27')
bucket_obj2 = bucket.objects.filter(Prefix='2022-01-28')
objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]

In [94]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR10.csv'),
 s3.Object

In [95]:
# Obtener el objeto por posición en el bucket objects
csv_obj_init = bucket.Object(key=objects[34].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [96]:
df_init

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE0005933931,EXS1,ISHS CORE DAX UC.ETF EOA,ETF,EUR,2504265,2022-01-28,10:00,129.620,129.620,129.620,129.620,182,1
1,LU0292106241,DXSN,XTR.SHORTDAX DAILY SW. 1C,ETF,EUR,2504270,2022-01-28,10:00,13.954,13.954,13.954,13.954,2412,5
2,LU0411075020,DBPD,XTR.SHORTDAX X2 DA.SW. 1C,ETF,EUR,2504272,2022-01-28,10:00,1.419,1.419,1.419,1.419,81600,1
3,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-28,10:00,13.020,13.020,13.010,13.010,2354,4
4,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-01-28,10:00,184.350,184.350,184.300,184.300,55,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12802,DE000A3GN5J9,ELTC,ETC ISSUANCE O.END ETN,ETN,EUR,6267071,2022-01-28,10:59,9.405,9.405,9.405,9.405,277,1
12803,DE000A3CNK42,YOU,ABOUT YOU HOLDING SE,Common stock,EUR,6501749,2022-01-28,10:59,16.560,16.600,16.560,16.600,206,4
12804,DE000A3E5D64,FPE3,FUCHS PETROLUB VZO NA ON,Common stock,EUR,6699158,2022-01-28,10:59,37.580,37.620,37.580,37.620,152,2
12805,FR0004056851,AYJ,"VALNEVA SE EO -,15",Common stock,EUR,6769323,2022-01-28,10:59,14.870,14.870,14.870,14.870,7,1


### Convertir múltiples objetos a dataframe

In [97]:
# Se inicializa el nombre de las columnas extrayendo los nombres de df_init
df_all = pd.DataFrame(columns=df_init.columns)

# Corroboramos que el nombre se ha establecido
df_all.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

Concatenar objetos con pandas:

https://pandas.pydata.org/docs/reference/api/pandas.concat.html

In [98]:
# Concatenar objetos con pandas
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = pd.concat([df,df_all], ignore_index=True)

In [99]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000ETFL011,EL4A,DK DAX,ETF,EUR,2504258,2022-01-28,16:00,137.240,137.240,137.240,137.240,13,1
1,DE000ETFL060,EL4F,DK DAX (AUSSCHUETTEND),ETF,EUR,2504259,2022-01-28,16:00,64.630,64.630,64.630,64.630,49,1
2,DE0002635273,EXSB,ISHARES DIVDAX UCITS ETF,ETF,EUR,2504264,2022-01-28,16:00,20.510,20.510,20.510,20.510,100,1
3,DE0005933931,EXS1,ISHS CORE DAX UC.ETF EOA,ETF,EUR,2504265,2022-01-28,16:00,129.540,129.540,129.500,129.500,220,2
4,LU0252634307,LYY8,MUL-LYX.DLY.LEVDAX U.E. A,ETF,EUR,2504268,2022-01-28,16:00,130.520,130.520,130.480,130.480,158,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250379,DE000A3CNK42,YOU,ABOUT YOU HOLDING SE,Common stock,EUR,6501749,2022-01-27,08:59,16.560,16.580,16.560,16.580,456,5
250380,DE000A3E5D64,FPE3,FUCHS PETROLUB VZO NA ON,Common stock,EUR,6699158,2022-01-27,08:59,37.860,37.860,37.860,37.860,274,3
250381,CH0445689208,21XH,21SHARES HODL BSK ETP OE,ETN,EUR,6849646,2022-01-27,08:59,11.237,11.237,11.236,11.236,215,2
250382,IE00BLCHJN13,LI7U,GL X ETF-GXLIBATC DLA,ETF,EUR,7121507,2022-01-27,08:59,11.900,11.900,11.900,11.900,26,1


In [100]:
# Filtrar y extraer sólo algunas columnas
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [101]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,DE000ETFL011,2022-01-28,16:00,137.240,137.240,137.240,137.240,13
1,DE000ETFL060,2022-01-28,16:00,64.630,64.630,64.630,64.630,49
2,DE0002635273,2022-01-28,16:00,20.510,20.510,20.510,20.510,100
3,DE0005933931,2022-01-28,16:00,129.540,129.540,129.500,129.500,220
4,LU0252634307,2022-01-28,16:00,130.520,130.520,130.480,130.480,158
...,...,...,...,...,...,...,...,...
250379,DE000A3CNK42,2022-01-27,08:59,16.560,16.580,16.560,16.580,456
250380,DE000A3E5D64,2022-01-27,08:59,37.860,37.860,37.860,37.860,274
250381,CH0445689208,2022-01-27,08:59,11.237,11.237,11.236,11.236,215
250382,IE00BLCHJN13,2022-01-27,08:59,11.900,11.900,11.900,11.900,26


In [102]:
# Eliminar missing values
df_all.dropna(inplace=True)

In [103]:
# Consultar la forma del df para verificar si se borró alguna fila
df_all.shape

(250384, 8)

## Transformaciones

### Obtener precio de apertura por ISIN y día

In [110]:
df_sorted = df_all.sort_values(by=['Time'])
df_sorted

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
108596,AT0000A0E9W5,2022-01-28,08:00,16.39,16.44,16.39,16.44,3249,16.39
108738,DE000A2TSL71,2022-01-28,08:00,19.80,19.80,19.80,19.80,240,19.80
108739,LU2333210958,2022-01-28,08:00,28.80,28.80,28.60,28.60,2381,28.80
108740,DE000A3CNK42,2022-01-28,08:00,17.00,17.00,16.95,16.95,15217,17.00
108741,DE000A3E5D64,2022-01-28,08:00,38.14,38.26,38.14,38.18,396,38.14
...,...,...,...,...,...,...,...,...,...
125331,US8330341012,2022-01-27,20:30,184.20,184.20,184.20,184.20,0,184.20
125332,DE000A3E5ET8,2022-01-27,20:30,36.60,36.60,36.60,36.60,0,36.60
125333,GB0009390070,2022-01-27,20:30,3.48,3.48,3.48,3.48,0,3.48
125334,NL0012866412,2022-01-27,20:30,75.00,75.00,75.00,75.00,0,75.00


In [113]:
df_grouped = df_sorted.groupby(['ISIN', 'Date'])['StartPrice']
df_grouped

In [114]:
df_grouped.transform('first')

108596     16.39
108738     19.80
108739     28.80
108740     17.00
108741     38.14
           ...  
125331    184.20
125332     36.60
125333      3.48
125334     75.00
125335     42.00
Name: StartPrice, Length: 250384, dtype: float64

In [116]:
df_all['opening_price'] = df_grouped.transform('first')
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,DE000ETFL011,2022-01-28,16:00,137.240,137.240,137.240,137.240,13,138.300
1,DE000ETFL060,2022-01-28,16:00,64.630,64.630,64.630,64.630,49,65.120
2,DE0002635273,2022-01-28,16:00,20.510,20.510,20.510,20.510,100,20.535
3,DE0005933931,2022-01-28,16:00,129.540,129.540,129.500,129.500,220,130.560
4,LU0252634307,2022-01-28,16:00,130.520,130.520,130.480,130.480,158,132.480
...,...,...,...,...,...,...,...,...,...
250379,DE000A3CNK42,2022-01-27,08:59,16.560,16.580,16.560,16.580,456,16.210
250380,DE000A3E5D64,2022-01-27,08:59,37.860,37.860,37.860,37.860,274,37.520
250381,CH0445689208,2022-01-27,08:59,11.237,11.237,11.236,11.236,215,11.227
250382,IE00BLCHJN13,2022-01-27,08:59,11.900,11.900,11.900,11.900,26,11.916


In [77]:
# Todo el código anterior en una línea
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [107]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
845,AT0000A0E9W5,2022-01-28,16:02,16.20,16.20,16.20,16.20,256,16.39
1234,AT0000A0E9W5,2022-01-28,16:03,16.21,16.21,16.21,16.21,570,16.39
2130,AT0000A0E9W5,2022-01-28,16:06,16.21,16.21,16.21,16.21,628,16.39
2536,AT0000A0E9W5,2022-01-28,16:08,16.22,16.23,16.22,16.23,1041,16.39
3329,AT0000A0E9W5,2022-01-28,16:11,16.25,16.27,16.25,16.25,5780,16.39
...,...,...,...,...,...,...,...,...,...
247632,AT0000A0E9W5,2022-01-27,08:48,16.52,16.53,16.52,16.53,661,16.35
248130,AT0000A0E9W5,2022-01-27,08:50,16.55,16.56,16.55,16.56,474,16.35
248838,AT0000A0E9W5,2022-01-27,08:53,16.56,16.56,16.51,16.51,489,16.35
249915,AT0000A0E9W5,2022-01-27,08:58,16.57,16.57,16.57,16.57,116,16.35


### Obtener el precio de cierre por ISIN y día

In [79]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [80]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
845,AT0000A0E9W5,2022-01-28,16:02,16.20,16.20,16.20,16.20,256,16.39,16.35
1234,AT0000A0E9W5,2022-01-28,16:03,16.21,16.21,16.21,16.21,570,16.39,16.35
2130,AT0000A0E9W5,2022-01-28,16:06,16.21,16.21,16.21,16.21,628,16.39,16.35
2536,AT0000A0E9W5,2022-01-28,16:08,16.22,16.23,16.22,16.23,1041,16.39,16.35
3329,AT0000A0E9W5,2022-01-28,16:11,16.25,16.27,16.25,16.25,5780,16.39,16.35
...,...,...,...,...,...,...,...,...,...,...
247632,AT0000A0E9W5,2022-01-27,08:48,16.52,16.53,16.52,16.53,661,16.35,16.54
248130,AT0000A0E9W5,2022-01-27,08:50,16.55,16.56,16.55,16.56,474,16.35,16.54
248838,AT0000A0E9W5,2022-01-27,08:53,16.56,16.56,16.51,16.51,489,16.35,16.54
249915,AT0000A0E9W5,2022-01-27,08:58,16.57,16.57,16.57,16.57,116,16.35,16.54


## Aggregations

In [81]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [82]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-01-27,37.600,37.000,37.000,37.900,485
1,AT000000STR1,2022-01-28,38.050,37.000,37.000,38.050,456
2,AT00000FACC2,2022-01-27,7.640,7.620,7.620,7.640,60
3,AT00000FACC2,2022-01-28,7.660,7.520,7.520,7.660,610
4,AT0000606306,2022-01-27,24.160,25.080,24.160,25.260,1363
...,...,...,...,...,...,...,...
6360,XS2314659447,2022-01-28,8.774,8.576,8.570,8.774,1037
6361,XS2314660700,2022-01-27,20.206,20.316,20.206,20.472,148
6362,XS2314660700,2022-01-28,20.328,20.246,20.054,20.328,58
6363,XS2376095068,2022-01-27,32.216,32.884,32.216,33.326,11504


### Porcentaje Cambio Anterior Cierre

In [83]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [84]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT000000STR1,2022-01-27,37.600,37.000,37.000,37.900,485,NaN
1,AT000000STR1,2022-01-28,38.050,37.000,37.000,38.050,456,37.0000
2,AT00000FACC2,2022-01-27,7.640,7.620,7.620,7.640,60,NaN
3,AT00000FACC2,2022-01-28,7.660,7.520,7.520,7.660,610,7.6200
4,AT0000606306,2022-01-27,24.160,25.080,24.160,25.260,1363,NaN
...,...,...,...,...,...,...,...,...
6360,XS2314659447,2022-01-28,8.774,8.576,8.570,8.774,1037,8.8155
6361,XS2314660700,2022-01-27,20.206,20.316,20.206,20.472,148,NaN
6362,XS2314660700,2022-01-28,20.328,20.246,20.054,20.328,58,20.3160
6363,XS2376095068,2022-01-27,32.216,32.884,32.216,33.326,11504,NaN


In [85]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [86]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [87]:
df_all = df_all.round(decimals=2)

In [88]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-27,37.60,37.00,37.00,37.90,485,NaN
1,AT000000STR1,2022-01-28,38.05,37.00,37.00,38.05,456,0.00
2,AT00000FACC2,2022-01-27,7.64,7.62,7.62,7.64,60,NaN
3,AT00000FACC2,2022-01-28,7.66,7.52,7.52,7.66,610,-1.31
4,AT0000606306,2022-01-27,24.16,25.08,24.16,25.26,1363,NaN
...,...,...,...,...,...,...,...,...
6360,XS2314659447,2022-01-28,8.77,8.58,8.57,8.77,1037,-2.72
6361,XS2314660700,2022-01-27,20.21,20.32,20.21,20.47,148,NaN
6362,XS2314660700,2022-01-28,20.33,20.25,20.05,20.33,58,-0.34
6363,XS2376095068,2022-01-27,32.22,32.88,32.22,33.33,11504,NaN
